In [36]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd

import optuna
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [37]:
# Mock dataset for demonstration
class TextDataset(Dataset):
    # def __init__(self, vocabulary_size, sequence_length, num_samples):
    #     self.data = torch.randint(0, vocabulary_size, (num_samples, sequence_length))
    #     self.labels = torch.randint(0, 2, (num_samples,))
    def __init__(self, texts, labels, sequence_length):
        self.texts = texts
        self.labels = labels
        self.sequence_length = sequence_length
        self.vocab = self.build_vocab(texts)
        self.encoded_texts = [self.encode_text(text) for text in texts]

    def build_vocab(self, texts):
        unique_words = set(word for text in texts for word in text.lower().split())
        vocab = {word: i + 1 for i, word in enumerate(unique_words)}  # +1 for padding token at index 0
        return vocab
    
    def encode_text(self, text):
        return [self.vocab.get(word, 0) for word in text.lower().split()][:self.sequence_length] + [0] * (self.sequence_length - len(text.split()))

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return torch.tensor(self.encoded_texts[idx]), torch.tensor(self.labels[idx])
    

# Define the Q-network model
class QLearning(nn.Module):
    def __init__(self, vocabulary_size, embedding_dim, hidden_dim, dropout_rate=0.5):
        super(QLearning, self).__init__()
        self.embedding = nn.Embedding(vocabulary_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True, num_layers=2)
        self.dropout = nn.Dropout(dropout_rate)
        # Two actions: 0 or 1
        self.fc = nn.Linear(hidden_dim * 2, 2)  

    def forward(self, x):
        embeds = self.embedding(x)
        lstm_out, _ = self.lstm(embeds)
        out = self.dropout(lstm_out[:, -1])
        q_values = self.fc(out)
        return q_values

def update_q_values(model, optimizer, states, actions, rewards, next_states, gamma=0.99):
    model.eval()
    with torch.no_grad():
        q_values_next = model(next_states).max(1)[0]  # Get max Q value for next states
    model.train()
    q_values = model(states).gather(1, actions.unsqueeze(-1)).squeeze(-1)
    
    # Q-Learning update target
    target = rewards + gamma * q_values_next
    loss = nn.functional.mse_loss(q_values, target)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()

In [38]:
def train(model, device, train_loader, optimizer, epoch, gamma=0.99, log_interval=10):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        # Predict Q-values for current states (data)
        current_q_values = model(data)

        # Simulate taking actions (classifying) and receiving rewards
        # In a real RL scenario, actions are taken based on a policy. Here, we simplify.
        _, predicted_actions = torch.max(current_q_values, 1)
        rewards = (predicted_actions == target).float()  # Reward is 1 for correct classification, 0 otherwise

        # Simulate next states (in practice, you might use a different strategy)
        # For simplicity, let's just roll data to simulate "next states"
        next_data = torch.roll(data, -1, 0)
        with torch.no_grad():
            future_q_values = model(next_data).max(1)[0]  # Use max Q-value for simplicity
        
        # Compute target Q-values
        target_q_values = rewards + gamma * future_q_values

        # Update model based on the difference between current Q-values and target Q-values
        # Assuming binary classification, gather Q-values for the taken actions
        action_indices = target.view(-1, 1).long()
        gathered_q_values = current_q_values.gather(1, action_indices).squeeze()

        loss = nn.functional.mse_loss(gathered_q_values, target_q_values.detach())
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}'
                  f' ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

def validate(model, device, validation_loader):
    model.eval()
    validation_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in validation_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            validation_loss += nn.CrossEntropyLoss()(output, target).item()  # Sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    validation_loss /= len(validation_loader.dataset)
    validation_acc = correct / len(validation_loader.dataset)
    print(f'\nValidation set: Average loss: {validation_loss:.4f}, Accuracy: {correct}/{len(validation_loader.dataset)} ({100. * correct / len(validation_loader.dataset):.0f}%)\n')
    return validation_loss, validation_acc

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += nn.CrossEntropyLoss()(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')


In [39]:
# Example texts and labels
texts = [
    "Great movie loved it", "Did not like the movie", "Awesome plot and acting",
    "Boring and too long", "Wonderfully made film", "Not my cup of tea",
    "Thrilling and exciting from start to finish", "Could have been better", "A masterpiece", "Waste of time",
    "The director did a fantastic job", "Terrible performances", "The cinematography is gorgeous", 
    "I fell asleep halfway through", "An unforgettable journey", "I've seen better", 
    "A total letdown from a great director", "The soundtrack complemented the movie perfectly", 
    "Lacked depth in the storyline", "Characters were not developed well", "Laughed all the way through", 
    "Failed to deliver a compelling narrative", "Visual effects were stunning", "Plot was predictable", 
    "Highly recommend this movie", "Not worth your time", "An epic disappointment", 
    "Engaging from start to finish", "Script was beautifully written", "The movie felt rushed",
    "A visual masterpiece", "Forgettable experience", "Will watch it again", "Do not waste your money", 
    "The acting was subpar", "A story well told", "Lacks originality", "Captivating to the very end", 
    "Missed the mark on many levels", "One of the year's best films"
]
labels = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1]

# Splitting dataset into train+val and test
train_val_texts, test_texts, train_val_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Splitting train+val into train and val
train_texts, validation_texts, train_labels, validation_labels = train_test_split(train_val_texts, train_val_labels, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# Creating datasets
sequence_length = 10  # Max number of words in a text
train_dataset = TextDataset(train_texts, train_labels, sequence_length)
validation_dataset = TextDataset(validation_texts, validation_labels, sequence_length)
test_dataset = TextDataset(test_texts, test_labels, sequence_length)

# Creating DataLoaders
batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for data, label in train_loader:
    print(f"Validation Encoded text: {data}")
    print(f"Validation Label: {label}")
    break  # Just show one batch for brevity

for data, label in validation_loader:
    print(f"Validation Encoded text: {data}")
    print(f"Validation Label: {label}")
    break  # Just show one batch for brevity

for data, label in test_loader:
    print(f"Test Encoded text: {data}")
    print(f"Test Label: {label}")
    break  # Just show one batch for brevity

Validation Encoded text: tensor([[38, 20, 57, 24,  0,  0,  0,  0,  0,  0],
        [40,  8, 58, 29,  0,  0,  0,  0,  0,  0],
        [22, 18,  0,  0,  0,  0,  0,  0,  0,  0],
        [67, 50, 77, 76, 43,  2, 41,  0,  0,  0]])
Validation Label: tensor([1, 1, 0, 1])
Validation Encoded text: tensor([[17,  3, 29,  0,  0,  0,  0,  0,  0,  0],
        [15, 23, 21, 13,  0,  0,  0,  0,  0,  0],
        [15, 11, 16, 20, 10, 27,  0,  0,  0,  0],
        [ 8, 28,  4, 24,  0,  0,  0,  0,  0,  0]])
Validation Label: tensor([1, 0, 1, 1])
Test Encoded text: tensor([[22, 14, 27,  4, 21,  0,  0,  0,  0,  0],
        [25,  8, 13, 23, 25, 19, 12,  0,  0,  0],
        [18,  5, 11,  0,  0,  0,  0,  0,  0,  0],
        [20, 31, 29,  0,  0,  0,  0,  0,  0,  0]])
Test Label: tensor([0, 0, 0, 0])


In [40]:
# Parameters and Hyperparameters
vocabulary_size = 10000  # to adjust 
sequence_length = 50  # to adjust 
embedding_dim = 128
hidden_dim = 64
num_classes = 2
batch_size = 64
epochs = 10
learning_rate = 0.001

# Model, optimizer, and device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = QLearning(vocabulary_size, embedding_dim, hidden_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    validate(model, device, validation_loader)

# After training, evaluate on the test set
test(model, device, test_loader)

Train Epoch: 1 [0/24 (0%)]	Loss: 0.704901

Validation set: Average loss: 0.2658, Accuracy: 1/8 (12%)

Train Epoch: 2 [0/24 (0%)]	Loss: 0.723520

Validation set: Average loss: 0.3659, Accuracy: 1/8 (12%)

Train Epoch: 3 [0/24 (0%)]	Loss: 1.347499

Validation set: Average loss: 0.4732, Accuracy: 1/8 (12%)

Train Epoch: 4 [0/24 (0%)]	Loss: 0.720412

Validation set: Average loss: 0.5232, Accuracy: 1/8 (12%)

Train Epoch: 5 [0/24 (0%)]	Loss: 2.638391

Validation set: Average loss: 0.5440, Accuracy: 1/8 (12%)

Train Epoch: 6 [0/24 (0%)]	Loss: 2.393632

Validation set: Average loss: 0.5669, Accuracy: 1/8 (12%)

Train Epoch: 7 [0/24 (0%)]	Loss: 6.168314

Validation set: Average loss: 0.5986, Accuracy: 1/8 (12%)

Train Epoch: 8 [0/24 (0%)]	Loss: 3.854575

Validation set: Average loss: 0.5451, Accuracy: 1/8 (12%)

Train Epoch: 9 [0/24 (0%)]	Loss: 2.414689

Validation set: Average loss: 0.5277, Accuracy: 1/8 (12%)

Train Epoch: 10 [0/24 (0%)]	Loss: 1.326143

Validation set: Average loss: 0.5534, 

In [41]:
def objective(trial):
    # Define the search space
    vocabulary_size = trial.suggest_categorical('vocabulary_size', [5000, 10000, 20000, 40000])
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    embedding_dim = trial.suggest_categorical('embedding_dim', [64, 128, 256])
    hidden_dim = trial.suggest_categorical('hidden_dim', [32, 64, 128])
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'RMSprop', 'SGD'])
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)
    step_size = trial.suggest_int('step_size', 1, 100)
    gamma = trial.suggest_float('gamma', 0.1, 1.0, log=True)
    sequence_length = trial.suggest_categorical('sequence_length', [50, 100, 200, 400])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Device configuration
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Model setup with trial suggestions
    model = QLearning(vocabulary_size, embedding_dim, hidden_dim, dropout_rate=dropout_rate).to(device)

    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=learning_rate)

    if optimizer_name == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    elif optimizer_name == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
    elif optimizer_name == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

    # Training loop
    epochs = 10  # Reduced for faster optimization cycles
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        val_loss, val_accuracy = validate(model, device, validation_loader)
        scheduler.step()

    # Set custom attributes for the trial
    trial.set_user_attr("val_loss", val_loss)
    trial.set_user_attr("val_accuracy", val_accuracy)
    
    # print(f"Returning from validate: val_loss={val_loss}, val_accuracy={val_accuracy}")
    # return val_loss

    # Objective: maximize validation accuracy by minimizing its negative value
    return -val_accuracy  # Return the negative accuracy

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)  # Number of trials can be adjusted

print('Number of finished trials:', len(study.trials))
print('Best trial:')
trial = study.best_trial

# Retrieve the validation loss and accuracy from the best trial
best_val_loss = trial.user_attrs["val_loss"]
best_val_accuracy = trial.user_attrs["val_accuracy"]

print(f'Best Validation Loss: {best_val_loss}')
print(f'Best Validation Accuracy: {best_val_accuracy}')
print('Best Trial Parameters:')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

[I 2024-04-02 19:23:47,108] A new study created in memory with name: no-name-a329df8e-29b0-4ddb-990a-1d1c7a4721e7


Train Epoch: 1 [0/24 (0%)]	Loss: 0.582008

Validation set: Average loss: 0.1005, Accuracy: 1/8 (12%)

Train Epoch: 2 [0/24 (0%)]	Loss: 1.053831

Validation set: Average loss: 0.1238, Accuracy: 1/8 (12%)

Train Epoch: 3 [0/24 (0%)]	Loss: 1.637613

Validation set: Average loss: 0.1358, Accuracy: 1/8 (12%)

Train Epoch: 4 [0/24 (0%)]	Loss: 1.487227

Validation set: Average loss: 0.1561, Accuracy: 1/8 (12%)

Train Epoch: 5 [0/24 (0%)]	Loss: 3.251556

Validation set: Average loss: 0.1705, Accuracy: 1/8 (12%)

Train Epoch: 6 [0/24 (0%)]	Loss: 3.457337

Validation set: Average loss: 0.1674, Accuracy: 1/8 (12%)

Train Epoch: 7 [0/24 (0%)]	Loss: 2.743927

Validation set: Average loss: 0.2106, Accuracy: 1/8 (12%)



[I 2024-04-02 19:23:47,551] Trial 0 finished with value: -0.125 and parameters: {'vocabulary_size': 20000, 'batch_size': 64, 'learning_rate': 0.026141110093901368, 'embedding_dim': 128, 'hidden_dim': 64, 'optimizer': 'Adam', 'dropout_rate': 0.48611235345371456, 'step_size': 65, 'gamma': 0.7730245414210906, 'sequence_length': 400}. Best is trial 0 with value: -0.125.


Train Epoch: 8 [0/24 (0%)]	Loss: 4.330476

Validation set: Average loss: 0.2470, Accuracy: 1/8 (12%)

Train Epoch: 9 [0/24 (0%)]	Loss: 4.641228

Validation set: Average loss: 0.1787, Accuracy: 1/8 (12%)

Train Epoch: 10 [0/24 (0%)]	Loss: 7.701687

Validation set: Average loss: 0.1635, Accuracy: 1/8 (12%)

Train Epoch: 1 [0/24 (0%)]	Loss: 0.295429

Validation set: Average loss: 0.0842, Accuracy: 7/8 (88%)

Train Epoch: 2 [0/24 (0%)]	Loss: 0.335663

Validation set: Average loss: 0.0832, Accuracy: 7/8 (88%)

Train Epoch: 3 [0/24 (0%)]	Loss: 0.339655

Validation set: Average loss: 0.0822, Accuracy: 7/8 (88%)

Train Epoch: 4 [0/24 (0%)]	Loss: 0.350284


[I 2024-04-02 19:23:47,805] Trial 1 finished with value: -0.875 and parameters: {'vocabulary_size': 20000, 'batch_size': 32, 'learning_rate': 0.012582993738760546, 'embedding_dim': 128, 'hidden_dim': 128, 'optimizer': 'SGD', 'dropout_rate': 0.0956867395758132, 'step_size': 60, 'gamma': 0.3770545166146685, 'sequence_length': 100}. Best is trial 1 with value: -0.875.


...

Train Epoch: 5 [0/24 (0%)]	Loss: 0.396181

Validation set: Average loss: 0.0843, Accuracy: 7/8 (88%)

Train Epoch: 6 [0/24 (0%)]	Loss: 0.329020

Validation set: Average loss: 0.0843, Accuracy: 7/8 (88%)

Train Epoch: 7 [0/24 (0%)]	Loss: 0.481130

Validation set: Average loss: 0.0843, Accuracy: 7/8 (88%)

Train Epoch: 8 [0/24 (0%)]	Loss: 0.492956

Validation set: Average loss: 0.0843, Accuracy: 7/8 (88%)

Train Epoch: 9 [0/24 (0%)]	Loss: 0.427249

Validation set: Average loss: 0.0843, Accuracy: 7/8 (88%)

Train Epoch: 10 [0/24 (0%)]	Loss: 0.574395

Validation set: Average loss: 0.0843, Accuracy: 7/8 (88%)

Train Epoch: 1 [0/24 (0%)]	Loss: 0.668570

Validation set: Average loss: 0.1207, Accuracy: 1/8 (12%)

Train Epoch: 2 [0/24 (0%)]	Loss: 0.768952

Validation set: Average loss: 0.1469, Accuracy: 1/8 (12%)

Train Epoch: 3 [0/24 (0%)]	Loss: 0.941845

Validation set: Average loss: 0.1718, Accuracy: 1/8 (12%)

Train Epoch: 4 [0/24 (0%)]	Loss: 1.205963

Validation set: Average loss: 0.1956, 

[I 2024-04-02 19:23:54,706] Trial 28 finished with value: -0.125 and parameters: {'vocabulary_size': 20000, 'batch_size': 32, 'learning_rate': 0.0032086674881737763, 'embedding_dim': 256, 'hidden_dim': 64, 'optimizer': 'Adam', 'dropout_rate': 0.35573048167271415, 'step_size': 68, 'gamma': 0.21840165433340322, 'sequence_length': 100}. Best is trial 1 with value: -0.875.


Train Epoch: 7 [0/24 (0%)]	Loss: 1.959657

Validation set: Average loss: 0.2536, Accuracy: 1/8 (12%)

Train Epoch: 8 [0/24 (0%)]	Loss: 2.154592

Validation set: Average loss: 0.2658, Accuracy: 1/8 (12%)

Train Epoch: 9 [0/24 (0%)]	Loss: 2.547555

Validation set: Average loss: 0.2765, Accuracy: 1/8 (12%)

Train Epoch: 10 [0/24 (0%)]	Loss: 2.964395

Validation set: Average loss: 0.2859, Accuracy: 1/8 (12%)

Train Epoch: 1 [0/24 (0%)]	Loss: 0.708223

Validation set: Average loss: 0.5000, Accuracy: 1/8 (12%)

Train Epoch: 2 [0/24 (0%)]	Loss: 6.179146

Validation set: Average loss: 0.4800, Accuracy: 1/8 (12%)

Train Epoch: 3 [0/24 (0%)]	Loss: 4.753709

Validation set: Average loss: 0.0495, Accuracy: 7/8 (88%)

Train Epoch: 4 [0/24 (0%)]	Loss: 0.283171

Validation set: Average loss: 1.8739, Accuracy: 1/8 (12%)

Train Epoch: 5 [0/24 (0%)]	Loss: 86.933525

Validation set: Average loss: 0.1801, Accuracy: 7/8 (88%)

Train Epoch: 6 [0/24 (0%)]	Loss: 71.696533

Validation set: Average loss: 2.1135

[I 2024-04-02 19:23:55,473] Trial 29 finished with value: -0.875 and parameters: {'vocabulary_size': 20000, 'batch_size': 64, 'learning_rate': 0.024923589071051883, 'embedding_dim': 128, 'hidden_dim': 128, 'optimizer': 'RMSprop', 'dropout_rate': 9.595946850933512e-05, 'step_size': 53, 'gamma': 0.45096678094975357, 'sequence_length': 400}. Best is trial 1 with value: -0.875.


Number of finished trials: 30
Best trial:
Best Validation Loss: 0.076851487159729
Best Validation Accuracy: 0.875
Best Trial Parameters:
    vocabulary_size: 20000
    batch_size: 32
    learning_rate: 0.012582993738760546
    embedding_dim: 128
    hidden_dim: 128
    optimizer: SGD
    dropout_rate: 0.0956867395758132
    step_size: 60
    gamma: 0.3770545166146685
    sequence_length: 100
